<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 835ms/step - loss: 0.6803 - accuracy: 0.5572 - val_loss: 0.6893 - val_accuracy: 0.4341
Epoch 2/10
26/26 [==============================] - 20s 786ms/step - loss: 0.6460 - accuracy: 0.6740 - val_loss: 0.6245 - val_accuracy: 0.7659
Epoch 3/10
26/26 [==============================] - 20s 785ms/step - loss: 0.5940 - accuracy: 0.7409 - val_loss: 0.5656 - val_accuracy: 0.7268
Epoch 4/10
26/26 [==============================] - 20s 787ms/step - loss: 0.5322 - accuracy: 0.7701 - val_loss: 0.6367 - val_accuracy: 0.5951
Epoch 5/10
26/26 [==============================] - 21s 792ms/step - loss: 0.4791 - accuracy: 0.7786 - val_loss: 0.5324 - val_accuracy: 0.7366
Epoch 6/10
26/26 [==============================] - 20s 785ms/step - loss: 0.4389 - accuracy: 0.8102 - val_loss: 0.3962 - val_accuracy: 0.8683
Epoch 7/10
26/26 [==============================] - 20s 781ms/step - loss: 0.4047 - accuracy: 0.8285 - val_loss: 0.3567 - val_accuracy: 0.8585

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1483), started 0:07:40 ago. (Use '!kill 1483' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.70.h5
26/26 - 21s - loss: 0.6786 - accuracy: 0.5572 - val_loss: 0.7044 - val_accuracy: 0.4341
Epoch 2/5

Epoch 00002: saving model to weights.02-0.66.h5
26/26 - 20s - loss: 0.6446 - accuracy: 0.6326 - val_loss: 0.6619 - val_accuracy: 0.4927
Epoch 3/5

Epoch 00003: saving model to weights.03-0.57.h5
26/26 - 21s - loss: 0.6044 - accuracy: 0.7153 - val_loss: 0.5696 - val_accuracy: 0.8000
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 20s - loss: 0.5525 - accuracy: 0.7506 - val_loss: 0.5274 - val_accuracy: 0.7512
Epoch 5/5

Epoch 00005: saving model to weights.05-0.47.h5
26/26 - 20s - loss: 0.5213 - accuracy: 0.7555 - val_loss: 0.4671 - val_accuracy: 0.8146


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 22s - loss: 0.6657 - accuracy: 0.6095 - val_loss: 0.6526 - val_accuracy: 0.6488


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6745 - accuracy: 0.5608 - val_loss: 0.6688 - val_accuracy: 0.4927
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6382 - accuracy: 0.6971 - val_loss: 0.6113 - val_accuracy: 0.8244


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6726 - accuracy: 0.5803 - val_loss: 0.6651 - val_accuracy: 0.5171
Epoch 2/50
26/26 - 21s - loss: 0.6369 - accuracy: 0.6776 - val_loss: 0.6207 - val_accuracy: 0.6976
Epoch 3/50
26/26 - 21s - loss: 0.5821 - accuracy: 0.7567 - val_loss: 0.5411 - val_accuracy: 0.8537
Epoch 4/50
26/26 - 20s - loss: 0.5480 - accuracy: 0.7494 - val_loss: 0.5128 - val_accuracy: 0.7561
Epoch 5/50
26/26 - 21s - loss: 0.4657 - accuracy: 0.8200 - val_loss: 0.3952 - val_accuracy: 0.9366
Epoch 6/50
26/26 - 21s - loss: 0.4100 - accuracy: 0.8418 - val_loss: 0.3700 - val_accuracy: 0.8683
Epoch 7/50
26/26 - 21s - loss: 0.3086 - accuracy: 0.9112 - val_loss: 0.2640 - val_accuracy: 0.9317
Epoch 8/50
26/26 - 21s - loss: 0.2532 - accuracy: 0.9185 - val_loss: 0.1815 - val_accuracy: 0.9707
Epoch 9/50
26/26 - 20s - loss: 0.2079 - accuracy: 0.9416 - val_loss: 0.1562 - val_accuracy: 0.9756
Epoch 10/50
26/26 - 21s - loss: 0.1568 - accuracy: 0.9501 - val_loss: 0.1401 - val_accuracy: 0.9561
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 816ms/step - loss: 0.6640 - accuracy: 0.5998 - val_loss: 0.6499 - val_accuracy: 0.5463
Epoch 2/5
26/26 [==============================] - 20s 786ms/step - loss: 0.6006 - accuracy: 0.7226 - val_loss: 0.5565 - val_accuracy: 0.8098
Epoch 3/5
26/26 [==============================] - 21s 792ms/step - loss: 0.5201 - accuracy: 0.8017 - val_loss: 0.5186 - val_accuracy: 0.8000
Epoch 4/5
26/26 [==============================] - 21s 807ms/step - loss: 0.4583 - accuracy: 0.8066 - val_loss: 0.5224 - val_accuracy: 0.7268
Epoch 5/5
26/26 [==============================] - 20s 788ms/step - loss: 0.3772 - accuracy: 0.8650 - val_loss: 0.5806 - val_accuracy: 0.6732


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.599757,0.664397,0.546342,0.649877
1,1,0.722628,0.601313,0.809756,0.556504
2,2,0.801703,0.520628,0.800000,0.518604
3,3,0.806569,0.458694,0.726829,0.522425
4,4,0.864964,0.375195,0.673171,0.580613


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 32s 1s/step - loss: 0.6691 - accuracy: 0.5791 - val_loss: 0.6552 - val_accuracy: 0.6927

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 792ms/step - loss: 0.6378 - accuracy: 0.7068 - val_loss: 0.6524 - val_accuracy: 0.6439

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6224 - accuracy: 0.7056 - val_loss: 0.6464 - val_accuracy: 0.6439

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 804ms/step - loss: 0.6144 - accuracy: 0.7165 - val_loss: 0.6363 - val_accuracy: 0.7122

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 796ms/step - loss: 0.6103

In [22]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 8639), started 0:17:16 ago. (Use '!kill 8639' to kill it.)

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 24s 905ms/step - loss: 0.6444 - accuracy: 0.6253 - val_loss: 0.6039 - val_accuracy: 0.7512
Epoch 2/50
26/26 [==============================] - 21s 801ms/step - loss: 0.5779 - accuracy: 0.6934 - val_loss: 0.5249 - val_accuracy: 0.7756
Epoch 3/50
26/26 [==============================] - 21s 812ms/step - loss: 0.4957 - accuracy: 0.7725 - val_loss: 0.4493 - val_accuracy: 0.8000
Epoch 4/50
26/26 [==============================] - 21s 795ms/step - loss: 0.4456 - accuracy: 0.8102 - val_loss: 0.3927 - val_accuracy: 0.8341
Epoch 5/50
25/26 [===========================>..] - ETA: 0s - loss: 0.3887 - accuracy: 0.8450
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 21s 789ms/step - loss: 0.3955 - accuracy: 0.8382 - val_loss: 0.4201 - val_accuracy: 0.8146
Epoch 6/50
26/26 [==============================] - 21s 792ms/step - loss: 0.3270 - accuracy: 0.8905 - val_loss: 0.3205 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 8639), started 0:17:16 ago. (Use '!kill 8639' to kill it.)